In [11]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [12]:
# Libraries
%load_ext autoreload
%autoreload 2
import index_toolkit as tk
import pandas as pd
import yfinance as yf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Inputs
market_input = '^FCHI'   #CAC 40
riskfree_input = '^TNX'     #10y US
index_df = pd.read_csv('portfolio_allocation.csv', index_col = 'Ticker') #Fichier à mofidier

In [14]:
# Fetching market_df data and risk-free rate
market_df = yf.Ticker(market_input).history('max')['Close']
market_df.name = market_input
market_df.index = pd.to_datetime(market_df.index, utc=True)
market_df = market_df.resample('B').last().ffill()
market_df.index = market_df.index.tz_localize(None)
riskfree = yf.Ticker(riskfree_input).history('1y')['Close'].iloc[-1] / 100

# Fetching and reating a DataFrame to store historical prices for each asset
history_df = pd.DataFrame()
for ticker in index_df.index:
    history = yf.Ticker(ticker).history('max')['Close']
    history.name = ticker
    history_df = pd.concat([history_df, history], axis=1)
history_df.index = pd.to_datetime(history_df.index, utc=True)
history_df = history_df.resample('B').last().ffill()
history_df.index = history_df.index.tz_localize(None)

In [15]:
print(f"Report's date: {history_df.index[-1].strftime("%d-%m-%Y")}")

Report's date: 27-09-2024


In [16]:
tk.index_table(index=index_df, history=history_df,).show()

interactive(children=(Select(description='by', options=('All', 'Overall', 'Type', 'Account', 'Industry'), valu…

In [17]:
tk.stats_table(index=index_df, history=history_df, market=market_df, riskfree=riskfree).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…

In [18]:
tk.risk_chart(index=index_df, history=history_df, market=market_df).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…

In [19]:
tk.chart_comparison(history=history_df, index=index_df, market=market_df).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…

In [20]:
tk.corr_table(index=index_df, history=history_df, market=market_df).show()

interactive(children=(Select(description='freq', index=1, options=('D', 'B', 'W', 'M'), value='B'), Select(des…